# Introduction/Abstract

Data set from King County Washington.

# Packages Needed/Setting Global Variables

In [32]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.formula.api import ols


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [33]:
def Decade(List):
    decade_list=[]
    oldest = List.min()
    i=0
    for x in List:
        i += 1
        while len(decade_list) < i:
            if (x >= oldest) & (x < (oldest+10)):
                decade_list.append('{}-{}'.format(oldest,(oldest+10)))
            oldest += 10
        oldest = List.min()
    return decade_list

In [34]:
def zoningLat(List):
    lowest = List.min()
    highest = List.max()
    distance = highest - lowest
    distance = math.ceil(distance*69)
    zones = []
    for entry in List:
        for zone in list(range(1,distance+1)):
            if ((lowest+(zone-1)/69) <= entry) &(entry < (lowest + zone/69)):
                zones.append(zone)
    return zones

def zoningLong(List):
    lowest = List.min()
    highest = List.max()
    distance = highest-lowest
    distance = math.ceil(distance*54.6)
    zones = []
    for entry in List:
        for zone in list(range(1,distance+1)):
            if ((lowest+(zone-1)/54.6) <= entry) & (entry < (lowest + zone/54.6)):
                zones.append(zone)
    return zones

In [35]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

# EDA

## Questions about the Data

## Import and Clean the Data

In [36]:
# Import the dataset
df = pd.read_csv('kc_house_data.csv')

#Dropping id column
df = df.drop(['id'],axis=1)

# Creating a new column for the year sold, dropping origianl date
df['year_sold'] = df.date.apply(lambda x: x[-4:]).astype(int)
df = df.drop(['date'], axis=1)

# Changing bedroom number for outlier, I think it was a typo of hitting 3 twice
# based off of the sqft
df.at[15856, 'bedrooms'] = 3

# Replacing NaN values in 'waterfront' with 0
df['waterfront'] = df['waterfront'].fillna(0)

# Replacing NaN values in view with the median/mode value which is 0 (63 entries)
# 0 means house was not viewed
df['view'] = df['view'].fillna(0)

# Replacing ? in sqft_basement and converting to float
df['sqft_basement'] = df['sqft_basement'].replace('?',0).astype(float)

# yr_renovated and renovated columns
df['yr_renovated'] = df['yr_renovated'].fillna(0)
df['Renovated'] = df['yr_renovated'] > 0
df['Renovated'] = df['Renovated'].astype(int)
df['yr_renovated'] = df['yr_renovated'].replace(0,df['yr_built'])

#Adding column for age when sold
df['Age_When_Sold'] = df['year_sold'] - df['yr_built']

#Adding column for years since renovation when sold and dropping yr_renovated
df['Age_Since_Renovation'] = df['year_sold'] - df['yr_renovated']

#Adding interaction variable Renovated*Age since renovation
df['Renovated*Age'] = df['Renovated'] * df['Age_Since_Renovation']

#Adding zones for Lat and Long
df['Lat_Zones'] = zoningLat(df['lat'])
df['Long_Zones'] = zoningLong(df['long'])
df['Lat_Zones'] = df['Lat_Zones'].astype(str)
df['Long_Zones'] = df['Long_Zones'].astype(str)
df['Zones'] = df['Lat_Zones'] + 'x' + df['Long_Zones']
df = df.drop(['lat','long','Long_Zones','Lat_Zones'],axis=1)

#Grouping year built by decade
df['yr_built'] = Decade(df.yr_built)

#Correlations that are getting dropped
df = df.drop(['yr_renovated'],axis=1)
df = df.drop(['Age_Since_Renovation'],axis=1)
df = df.drop(['sqft_living'],axis=1)
df = df.drop(['Renovated*Age'],axis=1)

In [37]:
#Creating Dummie Variables double check this list
dummy_list = ['bedrooms', 'bathrooms', 'floors', 'view', 'condition',
                'grade', 'yr_built', 'zipcode', 'year_sold',
                'Zones']
dum=df[dummy_list].astype('category')
dummies = pd.get_dummies(dum[dummy_list],prefix=dummy_list,drop_first=True)
df_preprocessed = df.drop(dummy_list, axis=1)
df_preprocessed = pd.concat([df_preprocessed,dummies],axis=1)

## Notes

# Modeling

In [38]:
y = df_preprocessed['price']
X = df_preprocessed.drop(['price'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [39]:
X_train_int = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_int).fit()
model.summary()
#Adj. R-squared values
#First run 0.853
#Second run 0.850 (after taking out yr_renovated)
#Third run 0.857 (after taking out Age_Since_Renovation)
#Fourth run 0.851 (after taking out sqft_living)
#Fifth run 0.851 (after taking out Renovated*Age)

/Users/stevenkyle/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/stevenkyle/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/stevenkyle/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/stevenkyle/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.884
Model:                            OLS   Adj. R-squared:                  0.877
Method:                 Least Squares   F-statistic:                     115.1
Date:                Mon, 19 Oct 2020   Prob (F-statistic):               0.00
Time:                        18:18:11   Log-Likelihood:            -2.2719e+05
No. Observations:               17277   AIC:                         4.565e+05
Df Residuals:                   16199   BIC:                         4.649e+05
Df Model:                        1077                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const              -9.001e+04    1.9e+05     -0.474      0.636   -4.62e+05    2.82e+05
sqft_lot               0.2589      0.041      6.387      0.000       0.179       0.338
waterfront          6.754e+05   1.68e+04     40.263      0.000    6.42e+05    7.08e+05
sqft_above           149.0809      3.048     48.914      0.000     143.107     155.055
sqft_basement        103.2180      3.392     30.427      0.000      96.569     109.867
sqft_living15         20.4915      2.978      6.882      0.000      14.655      26.328
sqft_lot15            -0.0030      0.094     -0.032      0.974      -0.186       0.180
Renovated           4.939e+04   6038.526      8.179      0.000    3.76e+04    6.12e+04
Age_When_Sold        106.9691    369.912      0.289      0.772    -618.099     832.037
bedrooms_2         -2416.8800   1.17e+04     -0.207      0.836   -2.53e+04    2.04e+04
bedrooms_3          1.007e+04   1.17e+04      0.859      0.391   -1.29e+04    3.31e+04
bedrooms_4           657.4405    1.2e+04      0.055      0.956   -2.29e+04    2.42e+04
bedrooms_5         -9454.6997   1.26e+04     -0.750      0.454   -3.42e+04    1.53e+04
bedrooms_6          -5.14e+04   1.54e+04     -3.336      0.001   -8.16e+04   -2.12e+04
bedrooms_7         -7.933e+04   2.67e+04     -2.969      0.003   -1.32e+05    -2.7e+04
bedrooms_8         -3.504e+04   3.94e+04     -0.890      0.374   -1.12e+05    4.21e+04
bedrooms_9         -1.723e+05   6.73e+04     -2.558      0.011   -3.04e+05   -4.03e+04
bedrooms_10        -2.484e+05   1.02e+05     -2.446      0.014   -4.47e+05   -4.93e+04
bedrooms_11        -1.242e-05   1.49e-05     -0.836      0.403   -4.16e-05    1.67e-05
bathrooms_0.75     -2.637e+04   7.84e+04     -0.336      0.737    -1.8e+05    1.27e+05
bathrooms_1.0       1.391e+04    7.6e+04      0.183      0.855   -1.35e+05    1.63e+05
bathrooms_1.25     -6.692e+04   9.58e+04     -0.699      0.485   -2.55e+05    1.21e+05
bathrooms_1.5       1.169e+04   7.61e+04      0.154      0.878   -1.37e+05    1.61e+05
bathrooms_1.75      9817.5022    7.6e+04      0.129      0.897   -1.39e+05    1.59e+05
bathrooms_2.0       8753.5882   7.61e+04      0.115      0.908    -1.4e+05    1.58e+05
bathrooms_2.25      2.906e+04   7.61e+04      0.382      0.703    -1.2e+05    1.78e+05
bathrooms_2.5       1.824e+04   7.61e+04      0.240      0.810   -1.31e+05    1.67e+05
bathrooms_2.75      1.455e+04   7.62e+04      0.191      0.849   -1.35e+05    1.64e+05
bathrooms_3.0       2.795e+04   7.62e+04      0.367      0.714   -1.21e+05    1.77e+05
bathrooms_3.25      6.333e+04   7.63e+04      0.830      0.407   -8.63e+04    2.13e+05
bathrooms_3.5       4.053e+04   7.63e+04      0.531      0.595   -1.09e+05     1.9e+05
bathrooms_3.75      7.207e+04    7.7e+04      0.936      0.349   -7.89e+04    2.23e+05
bathrooms_4.0        9.66e+04   7.72e+

In [11]:
continuous = ['sqft_lot', 'sqft_above',
              'sqft_basement', 'sqft_living15', 'sqft_lot15','Age_When_Sold']

In [9]:
#Remove Correlated Features
test = X_train#[continuous]

In [10]:
test_corr=test.corr().abs().stack().reset_index().sort_values(0, ascending=False)
test_corr['pairs'] = list(zip(test_corr.level_0, test_corr.level_1))
test_corr.set_index(['pairs'],inplace=True)
test_corr.drop(['level_0','level_1'],axis=1,inplace=True)
test_corr.columns = ['cc']
test_corr.drop_duplicates(inplace=True)
test_corr[(test_corr.cc >.75) & (test_corr.cc<1)]

,cc
pairs,
"(condition_3, condition_4)",0.813554


In [17]:
#Looks like yr_built and yr_renovated is highly correlated, dropping yr_renovated column
#Looks like Age_When_sold and Age_Since_Renovation is highly correlated, dropping Age_Since_Renovation
#Looks like sqft_living and sqft_above is highly correlated, dropping sqft_living
#Looks like Renovated*Age and Renovated is highly correlated, dropping Renovated*Age
#Looks like sqft_living and sqft_living15 is highly correlated, dropping sqft_living
#Also looks like condition_3 and condition_4 is highly correlated, not going to do anything for these since they are from the both category
#Did this in EDA section

In [40]:
results = stepwise_selection(X_train,y_train)

/Users/stevenkyle/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/stevenkyle/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/stevenkyle/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/stevenkyle/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/stevenkyle/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid valu

KeyboardInterrupt: 

In [ ]:
#Looking at scaling
test.hist(figsize=(7,10));